In [190]:
import numpy as np
from difflib import *
import os
import pandas as pd

In [191]:

import warnings
warnings.simplefilter('ignore')
# We will display plots right inside Jupyter Notebook
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import datetime

import re
import jieba
import mysql.connector
import matplotlib.font_manager as mfm

In [192]:
def get_connect():
    conn_temp = mysql.connector.connect(
            host = '139.162.122.115',#本地地址
            user = 'travel',#一般默认用户名
            passwd = 'ksd53890045',#本地数据库登录密码（这里用你自己的密码！！！）
            db = 'travelbigdata',#数据库名称（这里用你自己的数据库名称！！！）
            port = 3306,#安装mysql默认的端口号
            charset = 'utf8'#设置数据库统一编码  
    )
    return conn_temp

In [193]:
tablename='trip'
table={
    'id':'int(11) not null auto_increment PRIMARY KEY',
    'name':'varchar(100)',
    'img':'varchar(100)',
    'lat':'float',
    'lon':'float',
    'rating':'float',
    'review':'float',
    'price':'varchar(100)',
    'url':'varchar(1024)',
    'types':'varchar(1024)',
    'vicinity':'varchar(1024)',
    'web':'varchar(1024) ' ,
    'orderdetail':'varchar(1024) ' ,
    'other':'varchar(1024) ' ,
    'servies':'varchar(1024) ' ,
    'phone':'varchar(20) ' ,
    'googlepk':'int(11)',
    'area':'varchar(100) ' ,
    'city':'varchar(100) ' ,
    'create_at':'datetime   NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT "創建時間"',
    'modify_at':' datetime NOT  NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP COMMENT "修改時間"'
}
othersql='UNIQUE (url)'




In [194]:
def insetDB(tablename,table,data,conn_temp):
    cursor = conn_temp.cursor()
    sqltmpname=''
    sqltmpvalue=''
    for i in data:
        sqltmpname+=str(i)+' '+','
        if('char' in table[i]):
            sqltmpvalue+='"%s",' %data[i]
        else:
            sqltmpvalue+='%s,' %data[i]
    sqltmpname=sqltmpname[:-1]
    sqltmpvalue=sqltmpvalue[:-1]
    sqldata=[i for i in data]
    sqlInsert='insert into  '+tablename+'('+sqltmpname+') VALUES ('+sqltmpvalue+')'        
        
    
    sqlUpdate='UPDATE '+tablename+'  set rating=%s,review=%s where url= "%s" ' %\
        (data['rating'],data['review'],data['url'])
    
    
    try:
        cursor.execute(sqlInsert)
        conn_temp.commit()
    except:
        print('exsit   '+str(data['url']))
        cursor.execute(sqlUpdate)
        conn_temp.commit()

    return 

In [195]:
foodtypes=['餐廳', '餐館', '咖啡', '熟食', '冰品', '飲料', '餃子', '牛扒', '果汁', 
      '早餐', '酒吧',  '拉麵', '居酒屋', '酒店', '雪糕', '甜品', '麵店', '麵包', 
      '冷麵', '麵食店', '美式扒房',       '熟食店', '珍珠奶茶', '包點', '壽司', 
      '烏冬專門店', '菜館', '三文治店', '薄餅', '曲奇餅店',  '酒樓', '海味店', 
      '餡餅店', '快餐', '酒家', '飲料店', '茶館', '餅店', '健康食品專賣店','燒烤', 
      '小食店','餐車店','餐廳','茶藝']
nottypes=['用品店','批發商']

In [183]:
##################################
#
#
#
#
#
#
#
#
#
#
#
#
#
#
##################################

In [196]:
nameArea='Xinxing'

In [197]:
dataTrip=pd.read_csv('data/Trip/'+nameArea+'.csv')
dataTrip=dataTrip.drop_duplicates().reset_index(drop=True)

In [198]:
### 定義一下高雄區域，利用地址分類資料  ######
kao=['鼓山區','三民區','新興區','前金區','苓雅區','鹽埕區','前鎮區','旗津區','左營區','鳳山區','大寮區','大樹區','鳥松區','小港區','楠梓區','林園區','岡山區','仁武區','大社區','旗山區']
place='高雄'

In [199]:
area=[]
city=[]
for st,i in enumerate(dataTrip['vicinity']):
    areatmp=None
    placetmp=None
    for j in kao:
        if(str(j) in str(i)):
            areatmp=j
            placetmp=place
    if(pd.isnull(areatmp)):print(i)
    area.append(areatmp)
    city.append(placetmp)
dataTrip['area']=area
dataTrip['city']=city

成功一路76號
七賢一路311號 福華飯店六樓
林森一路272-1號
五福一路105號3樓
中山橫路1號
中正三路122號
六合一路148號 1樓
新田路168號
仁愛一街102號
林森一路176號
No.165 Linsen 1st Road
青年一路176巷12號
民享街97號
七賢二路92號
中山一路300號
中正三路73號
No. 90 Qixian 2nd Rd, Xinxing District, Kaohsiung City, Taiwan 800
新田路4號
自立二路158號
林森一路31號
六合二路
六合一路161號
玉竹一街17巷27號
文橫2路127巷13號
中山一路6之18號 高雄新崛江店
文化路18號
林森一路10號
尚義街129號
自立二路118號
五福二路26號
尚義街140號
民享街105號
五福二路262號 2樓
五福一路146號
新田路7號
民生一路54號


In [200]:
dataTrip.columns

Index(['img', 'lat', 'lon', 'name', 'order', 'other', 'phone', 'place',
       'price', 'rating', 'review', 'servies', 'types', 'url', 'vicinity',
       'web', 'where', 'area', 'city'],
      dtype='object')

In [201]:
conn_temp=get_connect()
for st in range(len(dataTrip)):
    if(st<=-1):
        pass
    else:
        rating='NULL' if pd.isna(dataTrip['rating'][st])else dataTrip['rating'][st]
        review='NULL' if pd.isna(dataTrip['review'][st])else dataTrip['review'][st]
        name=str(dataTrip['name'][st]).replace(',','').replace('|','').replace('、','')
        types=str(dataTrip['types'][st]).replace(',','_')
        price='NULL' if pd.isna(dataTrip['price'][st])else dataTrip['price'][st].replace(',','')
        phone=str(dataTrip['phone'][st]).replace(' ','')
        servies=str(dataTrip['servies'][st]).replace(',','')
        orderdetail=str(dataTrip['order'][st]).replace(',','')
        other=str(dataTrip['other'][st]).replace(',','')
        data={
            'name':name,
            'img':dataTrip['img'][st],
            'lat':dataTrip['lat'][st],
            'lon':dataTrip['lon'][st],
            'rating':rating,
            'review':review,
            'price':price,
            'url':dataTrip['url'][st],
            'types':types,
            'web':dataTrip['web'][st],
            'phone':phone,
            'vicinity':dataTrip['vicinity'][st],
            'googlepk':'NULL',
            'servies':servies,
            'orderdetail':orderdetail,
            'other':other,
            'area':dataTrip['area'][st],
            'city':dataTrip['city'][st]
        }
        insetDB('trip',table,data,conn_temp)
conn_temp.close()

exsit   https://www.tripadvisor.com.tw/Restaurant_Review-g13806674-d11810794-Reviews-Miche_House_Japanese_food_Restaurant-Lingya_Kaohsiung.html
exsit   https://www.tripadvisor.com.tw/Restaurant_Review-g13808513-d6952141-Reviews-Boston-Xinxing_Kaohsiung.html


In [31]:
conn_temp.close()